In [1]:
import pandas as pd
import torch
from preprocessing import *
from LSTM import LSTM
import time
set_seed(42)

In [2]:
df = pd.read_csv(r'output_csv\HCMUT-SuperNodeXP-2017-1.0.swf.csv')
# df = df.head(100)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15886 entries, 0 to 15885
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   job_id                    15886 non-null  float64
 1   submit_time               15886 non-null  float64
 2   wait_time                 15886 non-null  float64
 3   run_time                  15886 non-null  float64
 4   num_allocated_processors  15886 non-null  float64
 5   avg_cpu_time_used         15886 non-null  float64
 6   used_memory               15886 non-null  float64
 7   requested_processors      15886 non-null  float64
 8   requested_time            15886 non-null  float64
 9   requested_memory          15886 non-null  float64
 10  status                    15886 non-null  float64
 11  user_id                   15886 non-null  float64
 12  group_id                  15886 non-null  float64
 13  executable_id             15886 non-null  float64
 14  queue_

In [3]:
feature_columns = ['requested_processors', 'requested_time', 'avg_cpu_time_used', 'used_memory', 'submit_time', 'wait_time', 'user_id', 'group_id', 'executable_id', 'queue_id']
target_column = 'run_time'

# Hyperparameters
input_dim = len(feature_columns)
num_hidden = 64
num_layers = 3
dropout = 0.2
num_epochs = 10
lr = 0.001
batch_size = 128  
seq_len = 20
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataloaders, scaler = prepare_data_seq(df, feature_columns, target_column, seq_len=seq_len, batch_size=batch_size)

In [4]:
start = time.time()

# Initialize the model
model = LSTM(input_dim, num_hidden, num_layers, dropout).to(device)

# Train the model
model.train_model(model, dataloaders['train'], dataloaders['test'], num_epochs, lr, scaler)

end = time.time()

# Evaluate the model
rmse, mae, mse, r2 = model.evaluate_model(model, dataloaders['test'], scaler, input_dim)

print(f'\nFinal Test Metrics:')
print(f'Test RMSE: {rmse:.4f}, Test MAE: {mae:.4f}, Test MSE: {mse:.4f}, Test R2: {r2:.4f}')
print(f'Training Time: {end - start:.2f} seconds')

Epoch [1/10], Train Loss: 0.2697, Val Loss: 0.2945
Val RMSE: 137330.1949, Val MAE: 88173.0121, Val MSE: 18859582430.3554, Val R2: -0.1952
Model saved at epoch 1 with validation loss: 0.2945
Epoch [2/10], Train Loss: 0.2642, Val Loss: 0.3047
Val RMSE: 139810.7810, Val MAE: 89779.5161, Val MSE: 19547054497.0843, Val R2: -0.2388
Epoch [3/10], Train Loss: 0.2618, Val Loss: 0.3047
Val RMSE: 140109.8136, Val MAE: 89324.6984, Val MSE: 19630759873.8280, Val R2: -0.2441
Epoch [4/10], Train Loss: 0.2597, Val Loss: 0.2835
Val RMSE: 135604.4004, Val MAE: 84828.7821, Val MSE: 18388553396.8006, Val R2: -0.1654
Model saved at epoch 4 with validation loss: 0.2835
Epoch [5/10], Train Loss: 0.2552, Val Loss: 0.2837
Val RMSE: 134341.1771, Val MAE: 85182.1979, Val MSE: 18047551867.7351, Val R2: -0.1438
Epoch [6/10], Train Loss: 0.2473, Val Loss: 0.2802
Val RMSE: 130773.7931, Val MAE: 84368.2674, Val MSE: 17101784960.1524, Val R2: -0.0838
Model saved at epoch 6 with validation loss: 0.2802
Epoch [7/10], Tr